<a href="https://colab.research.google.com/github/jasonlee87/AIDL-Lab/blob/master/02_03_mnist_cnn_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tf.keras 패키지를 이용해 CNN(Convolution Neural Network) 만들기

1. 이미지 분류(Image Classification)를 위한 뉴럴 네트워크를(Neural Network)을 설계한다.
2. NN 모델을 훈련시킨다.
3. NN 모델의 정확도를 검증한다.

> TensorFlow 2 패키지를 설치하고 관련 모듈들을 import 한다.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model


> tf.keras.dataset의 mnist dataset을 다운받는다. int를 float로 변환 

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

> tf.data를 이용해 dataset을 섞는다.

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

> 이 예에서는 tf.keras의 Subclassing API 방식으로 CNN 모델을 만든다.

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

> 모델 훈련에 사용할 optimizer와 loss 함수를 선택한다. 

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

> 모델의 손실 및 정확도를 측정할 메트릭을 선택한다. 

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

> 모델 훈련을 위해 'tf.GradientTape'를 사용한다.
gradient(y, x)는 y를 x에 대해 미분한다라는 의미이다.

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
accu_plot = []
loss_plot = []

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  
  # storing the epoch end loss value to plot later
  accu_plot.append(train_accuracy.result())
  loss_plot.append(train_loss.result())

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

### CNN 기반 모델의 경우 MNIST 데이타셋 이미지 분류 정확도는 약 98% 수준이다.

> 모델 구조 요약

In [0]:
model.summary()

In [0]:
#tf.keras.utils.plot_model(model) 
tf.keras.utils.plot_model(model, 'my_first_model_with_shape_info.png', show_shapes=True)

In [0]:
!ls -al

In [0]:
epochs = range(1, len(accu_plot)+1)

plt.subplot(2, 1, 1)
plt.title('Accuray')
plt.plot(epochs, accu_plot, 'b', label='accuracy')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.9, 1.00])
plt.legend(loc='best')

plt.subplot(2, 1, 2)
plt.title('Cost')
plt.plot(epochs, loss_plot, 'r', label='loss')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.ylim([0.0, 0.2])
plt.legend(loc='best')
plt.show()
